In [ ]:
"""simple_circuit.ipynb"""
# Cell 1

from __future__ import annotations

import typing

import numpy as np
from IPython.core.display import Math

if typing.TYPE_CHECKING:
    from numpy.typing import NDArray


def display_array(
    a: NDArray[np.complex_], places: int = 5, column: bool = False, prefix: str = ""
) -> None:
    def strip(val: float) -> str:
        frmt: str = ":." + str(places) + "f"
        d: str = str("{v" + frmt + "}").format(v=val)
        while d[-1] == "0":
            d = d[:-1]
        if d[-1] == ".":
            d = d[:-1]
        if float(d) == 0:
            d = "0"
        return d

    m: NDArray[np.complex_] = np.copy(a)
    if len(m.shape) == 1:
        m = m[np.newaxis, :]
        if column:
            m = m.T
    prec: float = 1 / 10**places
    s: str = r"\begin{bmatrix}"
    for row in range(m.shape[0]):
        for col in range(m.shape[1]):
            v: np.complex_ = m[row, col]
            real_comp: float = float(np.real(v))
            imag_comp: float = float(np.imag(v))
            is_imag_neg: bool = imag_comp < 0
            is_real_zero: bool = bool(np.isclose(real_comp, 0, atol=prec))
            is_imag_zero: bool = bool(np.isclose(imag_comp, 0, atol=prec))
            is_imag_one: bool = bool(np.isclose(abs(imag_comp), 1, atol=prec))
            if is_real_zero:
                if is_imag_zero:
                    s += "0"
            else:
                s += strip(real_comp)
            if not is_imag_zero:
                if is_imag_one:
                    if is_imag_neg:
                        s += r"-i"
                    else:
                        if not is_real_zero:
                            s += "+"
                        s += r"i"
                else:
                    if not is_real_zero and not is_imag_neg:
                        s += " + "
                    s += strip(imag_comp) + "i"
            if col < m.shape[1] - 1:
                s += " &"
        s += r"\\"
    s += r"\end{bmatrix}"
    display(Math(prefix + s))


f: NDArray[np.complex_] = np.array([[1], [0]])
t: NDArray[np.complex_] = np.array([[0], [1]])

g_not: NDArray[np.complex_] = np.array([[0, 1], [1, 0]])
g_and: NDArray[np.complex_] = np.array([[1, 1, 1, 0], [0, 0, 0, 1]])
g_or: NDArray[np.complex_] = np.array([[1, 0, 0, 0], [0, 1, 1, 1]])

display_array(f, prefix=r"\mathbf{F}=0=")
display_array(t, prefix=r"\mathbf{T}=1=")

display_array(g_not, prefix=r"\mathbf{NOT}=")
display_array(g_and, prefix=r"\mathbf{AND}=")
display_array(g_or, prefix=r"\mathbf{OR}=")

In [ ]:
# Cell 2 - Implement Boolean circuit using matrix algebra


def circuit(
    a: NDArray[np.complex_],
    b: NDArray[np.complex_],
    c: NDArray[np.complex_],
    d: NDArray[np.complex_],
) -> NDArray[np.complex_]:
    g1: NDArray[np.complex_] = np.dot(g_and, np.kron(a, b))
    g2: NDArray[np.complex_] = np.dot(g_not, c)
    g3: NDArray[np.complex_] = np.dot(g_or, np.kron(g1, g2))
    g4: NDArray[np.complex_] = np.dot(g_and, np.kron(g3, d))
    g5: NDArray[np.complex_] = np.dot(g_not, g4)
    return g5


# Create truth table for three inputs and show the output
for a in [f, t]:
    for b in [f, t]:
        for c in [f, t]:
            for d in [f, t]:
                print(f"a: [{a[0][0]} {a[1][0]}]", end="  ")
                print(f"b: [{b[0][0]} {b[1][0]}]", end="  ")
                print(f"c: [{c[0][0]} {c[1][0]}]", end="  ")
                print(f"d: [{d[0][0]} {d[1][0]}]", end="  ")
                v: NDArray[np.complex_] = circuit(a, b, c, d)
                print(f"v: [{v[0][0]} {v[1][0]}]")